In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtsmixer import TSMixer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.training import train_tsmixer,train_latent_tsmixer,run_trials
from timeview.basis import BSplineBasis
from timeview.knot_selection import calculate_knot_placement

#add a path to the system path
import sys
sys.path.append('../')

from experiments.datasets import *
seed=42
torch.manual_seed(seed)

# Synthetic Datasets

In [2]:
sequence_length = 1
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=200
n_timesteps=prediction_length



Dataset_generator=SineTransDataset(n_samples=n_samples, n_timesteps=n_timesteps)

X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(X.values).float().reshape(n_samples,sequence_length,input_channels)
Y_torch=torch.tensor(Ys).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)


B=5


internal_knots=calculate_knot_placement(ts, ys, n_internal_knots=B-2, T=1,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)



mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([200, 1, 1]) torch.Size([200, 20, 1])


100%|██████████| 10/10 [00:35<00:00,  3.57s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0172, Std Test Loss: 0.0078


100%|██████████| 10/10 [01:00<00:00,  6.04s/it]

Mean Test Loss: 0.0174, Std Test Loss: 0.0081


In [3]:
sequence_length = 1
prediction_length = 20
input_channels = 2
output_channels = 1
n_samples=900
n_timesteps=prediction_length



Dataset_generator=BetaDataset(n_samples=n_samples, n_timesteps=n_timesteps)
X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(X.values).float().reshape(n_samples,sequence_length,input_channels)
Y_torch=torch.tensor(Ys).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)


B=5


internal_knots=calculate_knot_placement(ts, ys, n_internal_knots=B-2, T=1,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)



mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([900, 1, 2]) torch.Size([900, 20, 1])


100%|██████████| 10/10 [02:31<00:00, 15.11s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.1045, Std Test Loss: 0.0145


100%|██████████| 10/10 [04:44<00:00, 28.47s/it]

Mean Test Loss: 0.1089, Std Test Loss: 0.0147


In [4]:
sequence_length = 1
prediction_length = 20
input_channels = 4
output_channels = 1
n_samples=2000
n_timesteps=prediction_length


#generate a Tumor dataset
args={
'n_samples': n_samples,'n_time_steps': n_timesteps,'time_horizon': 1.0,'noise_std': 0.0,'seed': 0,'equation': 'wilkerson'
}

Dataset_generator=SyntheticTumorDataset(**args)

X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(X.values).float().reshape(n_samples,sequence_length,input_channels)
Y_torch=torch.tensor(Ys).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)


B=9


internal_knots=calculate_knot_placement(ts, ys, n_internal_knots=B-2, T=1,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)



mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)


torch.Size([2000, 1, 4]) torch.Size([2000, 20, 1])


100%|██████████| 10/10 [05:22<00:00, 32.23s/it]


Mean Test Loss: 0.0115, Std Test Loss: 0.0026


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [10:28<00:00, 62.85s/it]

Mean Test Loss: 0.0126, Std Test Loss: 0.0029


# Real Datasets

In [ ]:
os.chdir('../experiments')
Dataset_generator=FLChainDataset()
X,ts,ys=Dataset_generator.get_X_ts_ys()
X['sex'] = X['sex'].replace({'F': 0, 'M': 1})
X['mgus'] = X['mgus'].replace({'no': 0, 'yes': 1})

sequence_length = 1
prediction_length = 20
input_channels = X.shape[-1]
output_channels = 1

Ys=np.array(ys)
Ts=np.array(ts)


X_torch=torch.tensor(X.values).float().reshape(-1,sequence_length,input_channels)
Y_torch=torch.tensor(Ys).float().reshape(-1,prediction_length,output_channels)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, T=1.0, n_internal_knots=B-2, seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([5499, 1, 7]) torch.Size([5499, 20, 1])


100%|██████████| 10/10 [10:14<00:00, 61.45s/it]


Mean Test Loss: 0.0100, Std Test Loss: 0.0007


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [20:17<00:00, 121.80s/it]

Mean Test Loss: 0.0101, Std Test Loss: 0.0008


In [13]:
Dataset_generator=StressStrainDataset(**{
                "lot": "all",
                "include_lot_as_feature": True,
                "downsample": True,
                "more_samples": 0,
                "specimen": "all",
                "max_strain": 0.2})
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 1
prediction_length = 212

Ys=np.array([y[:prediction_length] for y in ys if len(y)>=prediction_length])
Ts=np.array([t[:prediction_length] for t in ts if len(t)>=prediction_length])
X=X[X.index.isin([i for i in range(len(ys)) if len(ys[i])>=prediction_length])]

transformer=Dataset_generator.get_default_column_transformer(keep_categorical=False)
X_transformed = transformer.fit_transform(X)

input_channels = X_transformed.shape[-1]
output_channels = 1


X_torch=torch.tensor(X_transformed).float().reshape(-1,sequence_length,input_channels)
Y_torch=torch.tensor(Ys).float().reshape(-1,prediction_length,output_channels)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr for arr in Ts]
ys_N=[arr for arr in Ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0,1),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([46, 1, 10]) torch.Size([46, 212, 1])


100%|██████████| 10/10 [00:12<00:00,  1.25s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0055, Std Test Loss: 0.0041


100%|██████████| 10/10 [00:16<00:00,  1.65s/it]

Mean Test Loss: 0.0064, Std Test Loss: 0.0029
